In [16]:
import requests
from zipfile import ZipFile
from urllib.request import urlretrieve
from boto3.session import Session
import boto3
import io
import os
from io import StringIO
import botocore
import pandas as pd
import numpy as np
from datetime import datetime
from pytz import timezone
import pytz
import os
import pyodbc
import struct
import datetime
import matplotlib.pyplot as plt
import scipy.stats as stats
from functools import reduce
from dotenv import load_dotenv
from functools import reduce
load_dotenv()
pd.options.mode.chained_assignment = None

In [17]:
def get_s3_object(bucket,folder,fileName):
    s3 = boto3.client('s3')
    key = folder+fileName
    obj = s3.get_object(Bucket=bucket, Key=key)
    df = pd.read_csv(obj['Body'],index_col=0)
    print(f'Downloaded: {bucket}:{key}')
    return df

In [18]:
#takes raw data from FRD in s3 and converts it into bar format
def get_formatted_future(bucket,folder,fileName):
    s3 = boto3.client('s3')
    key = folder+fileName
    obj = s3.get_object(Bucket=bucket, Key=key)
    df = pd.read_csv(obj['Body'],names=['Date', 'Open', 'High', 'Low', 'Close', 'Volume'])
    df = df.set_index('Date')
    df.index = pd.to_datetime(df.index)
    df.index = df.index.tz_localize('US/Eastern')
    df.index = df.index.strftime('%Y-%m-%d %H:%M:%S %z').str[:-2] + ':00'
    df['Engine'] = None
    df['Source'] = 'FirstRateData'
    df['Update'] = get_pst_time()
    return df

In [19]:
def check_if_s3_object_exists(bucket, folder,fileName):
    s3 = boto3.resource('s3')
    try:
        s3.Object(bucket, folder+fileName).load()
    except botocore.exceptions.ClientError as e:
        if e.response['Error']['Code'] == "404":
            # The object does not exist.
            return False
        else:
            # Something else has gone wrong.
            raise
    else:
        return True

In [20]:
def get_pst_time():
    date_format='%Y-%m-%d %H:%M:%S %z'
    date = datetime.datetime.now(tz=pytz.utc)
    date = date.astimezone(timezone('US/Pacific'))
    pstDateTime=date.strftime(date_format)
    pstDateTime = "{0}:{1}".format(pstDateTime[:-2],pstDateTime[-2:])
    return pstDateTime

In [21]:
def save_s3_object(bucket,folder,filename,datafrane):
    csv_buffer = StringIO()
    datafrane.to_csv(csv_buffer, sep=",", index=True)
    s3_resource = boto3.resource('s3')
    s3_resource.Object(bucket, folder+filename).put(Body=csv_buffer.getvalue())   
    print(f'Saved S3 Object: {bucket}:{folder}{filename}')

In [22]:
def update_2_hour(df, df2):
    df_slice = df.loc[df.index >= df2.index[-2]]
    df_format = get_2_hour(df_slice)
    return pd.concat([df2.iloc[:-2], df_format])

def get_2_hour(df_origin):
    df_ret = pd.DataFrame()
    counter = 0
    #turns every 2 rows into a row, except when there is only one row left
    while not df_origin.empty:
        if df_origin.shape[0] == 1:
            df_ret = df_ret.append(df_origin, ignore_index=True)
            df_ret.loc[df_ret.index[-1], 'Date'] = df_origin.index
            print(f'{counter} frames added')
            break
        else:
            df = df_origin.iloc[:2]
            new_row = {'Date':df.index[0],'Open':df['Open'][0],'High':df['High'].max(),'Low':df['Low'].min(),'Close':df['Close'][1],'Volume':df['Volume'].sum()}
            df_ret = df_ret.append(new_row, ignore_index=True)
            df_origin = df_origin.iloc[2:, :]
            counter += 1
            if counter % 10000 == 0:
                print(f'{counter} frames added')
    df_ret['Engine'] = 'Internal'
    df_ret['Source'] = 'FirstRateData'
    df_ret['Update'] = get_pst_time()
    return df_ret.set_index('Date')

In [23]:
#Daily bar consists of previous data from 18:00 from previous day -> 16:00 of next day
#this function takes the hourly data from FRD and converts it into daily data
def get_daily_bars(df):
    df_return = pd.DataFrame(columns=['Date','Open','High','Low','Close','Volume','Engine','Source','Update'])
    df = df.reset_index()
    datetime = df['Date'].str.split(" ", expand = True)
    df['Date'] = datetime[0]
    df['Time'] = datetime[1]
    df['Offset'] = datetime[2]
    #day for futures starts at 18:00:00, ends at 16:00:00
    df = df[df['Time'] != '17:00:00']
    df = df.reset_index(drop=True)
    prev, today, last_date = get_day(df)
    df_return = df_return.append(today, ignore_index=True)
    df_new = df.iloc[last_date+1:]
    while not df_new.empty:
        prev2, today2, last_date = get_day(df_new)
        if prev is None and today2 is not None:
            df_return = df_return.append(today2, ignore_index=True)
        elif today2 is None and prev is not None:
            df_return = df_return.append(prev, ignore_index=True)
        elif prev is not None and today2 is not None:
            new_row = concat_prev_today(prev,today2)
            df_return = df_return.append(new_row, ignore_index=True)
        prev = prev2
        df_new = df.iloc[last_date+1:]
    df_return['Engine'] = 'Internal'
    df_return['Source'] = 'FirstRateData'
    df_return['Update'] = get_pst_time()
    return df_return.set_index('Date')

def get_day(df):
    date = df.iloc[0]['Date']
    df_slice = df[df['Date'] == date]
    df_prev = df_slice[df_slice['Time'] > '17:00:00']
    df_today = df_slice[df_slice['Time'] < '17:00:00']
    prev = None
    today = None

    if not df_prev.empty:
        prev = format_df(df_prev)
    if not df_today.empty:    
        today = format_df(df_today)
    return prev, today, df_slice.index[-1]

In [24]:
#concats dataframe of rows and returns it as one row
def format_df(df):
    open_df = df['Open'].iloc[0]
    high_df = df['High'].max()
    low_df = df['Low'].min()
    close_df = df['Close'].iloc[-1]
    volume_df = df['Volume'].sum()
    date_df = df['Date'].iloc[0] + ' ' + df['Time'].iloc[0] + ' ' + df['Offset'].iloc[0]
    new_row = {'Date':date_df,'Open':open_df,'High':high_df,'Low':low_df,'Close':close_df,'Volume':volume_df}
    return new_row

In [25]:
#gets previous day's bar and next day's bar and concats it into one bar
def concat_prev_today(prev, today):
    new_row = {'Date':prev['Date'],'Open':prev['Open'],'High':max(prev['High'], today['High']),'Low':min(prev['Low'],today['Low']),'Close':today['Close'],'Volume':prev['Volume']+today['Volume'] }
    return new_row

Database Connection

In [26]:
server = 'prod.db.liminalcap.com'
database = 'LCData'
username = 'acameron'
password = 'dougal88'
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

In [28]:
#dict that stores differences between database code and FRD code
code_dict = {
    'A6': 'AD', 'AD': 'CD','B6': 'BP', 'BTP': 'IK','BZ': 'BZA',
    'CA': 'CC','DY': 'GX','E1': 'SF','E6': 'EC','EW': 'FA',
    'FX': 'VG','GF': 'FC','GG': 'RX','HE': 'LH','HR': 'OE',
    'J1': 'JY','LBS': 'LB','LE': 'LC','MES': 'HWA','MME': 'MES',
    'MNQ': 'HWB','MP': 'ME','MX': 'CF','N6': 'NV','RB': 'XB',
    'TN': 'ZT','VX': 'UX','YM': 'DM','ZC': 'C','ZF': 'TU',
    'ZL': 'BO','ZM': 'SM','ZN': 'TY','ZO': 'O','ZQ': 'FF',
    'ZR': 'RR','ZS': 'S','ZT': 'TU','ZW': 'W',
}

In [29]:
#gets raw files that are stored in s3 from FRD, then converts into bar format and stores back into s3

bucket = 'lc-tempstorage'
prefix = 'onemin/onemin/'
s3 = boto3.resource('s3')
my_bucket = s3.Bucket(bucket)

for obj in my_bucket.objects.filter(Prefix=prefix):
    filename = obj.key.split('/')[2]
    #df = get_formatted_future(bucket, prefix, filename)
    
    code = filename.split('_')[0]
    code_format = code_dict.get(code, code)
    #print(f'{code} : {code_format}')
    cursor.execute("SELECT [MinuteRecordId] FROM dbo.FuturesStubs WHERE BBSymbol = ? AND TTSymbol IS NOT NULL", code_format)
    for row in cursor.fetchall():
        cursor.execute("SELECT [S3BucketName], [S3PathName], [S3FileName] FROM dbo.DataRecords WHERE Id = ?", row[0])
        for row_nest in cursor.fetchall():
            if cursor.rowcount != 0:
                df = get_formatted_future(bucket, prefix, filename)
                save_s3_object(row_nest[0], row_nest[1], 'FRD_'+row_nest[2], df)

KeyboardInterrupt: 

In [ ]:
cursor.execute("SELECT [HourRecordId] FROM dbo.FuturesStubs WHERE TTSymbol IS NOT NULL")
for row in cursor.fetchall():
    cursor.execute("SELECT [S3BucketName], [S3PathName], [S3FileName] FROM dbo.DataRecords WHERE Id = ?", row[0])
    for row_nest in cursor.fetchall():
        twohour = 'FRD_'+row_nest[2].replace('hour', '2hour')
        if check_if_s3_object_exists(row_nest[0], row_nest[1], twohour):
            df = get_s3_object(row_nest[0], row_nest[1], twohour)
            df = df[['Open', 'High', 'Low', 'Close', 'Engine', 'Source', 'Update']]
            save_s3_object(row_nest[0], row_nest[1], twohour, df)

Downloaded: lc-market-data:Futures/US/CME/ES/FRD_ESC1_2hour.txt
Saved S3 Object: lc-market-data:Futures/US/CME/ES/FRD_ESC1_2hour.txt
Downloaded: lc-market-data:Futures/US/CME/NQ/FRD_NQC1_2hour.txt
Saved S3 Object: lc-market-data:Futures/US/CME/NQ/FRD_NQC1_2hour.txt
Downloaded: lc-market-data:Futures/US/CBT/DM/FRD_DMC1_2hour.txt
Saved S3 Object: lc-market-data:Futures/US/CBT/DM/FRD_DMC1_2hour.txt
Downloaded: lc-market-data:Futures/US/CME/FA/FRD_FAC1_2hour.txt
Saved S3 Object: lc-market-data:Futures/US/CME/FA/FRD_FAC1_2hour.txt
Downloaded: lc-market-data:Futures/US/CME/RTY/FRD_RTYC1_2hour.txt
Saved S3 Object: lc-market-data:Futures/US/CME/RTY/FRD_RTYC1_2hour.txt
Downloaded: lc-market-data:Futures/US/NYM/CL/FRD_CLC1_2hour.txt
Saved S3 Object: lc-market-data:Futures/US/NYM/CL/FRD_CLC1_2hour.txt
Downloaded: lc-market-data:Futures/US/NYM/NG/FRD_NGC1_2hour.txt
Saved S3 Object: lc-market-data:Futures/US/NYM/NG/FRD_NGC1_2hour.txt
Downloaded: lc-market-data:Futures/US/NYM/XB/FRD_XBC1_2hour.txt
S

In [ ]:
cursor.execute("SELECT [HourRecordId] FROM dbo.FuturesStubs WHERE TTSymbol IS NOT NULL")
for row in cursor.fetchall():
    cursor.execute("SELECT [S3BucketName], [S3PathName], [S3FileName] FROM dbo.DataRecords WHERE Id = ?", row[0])
    for row_nest in cursor.fetchall():
        twohour = 'FRD_'+row_nest[2].replace('hour', '2hour')
        if check_if_s3_object_exists(row_nest[0], row_nest[1], 'FRD_'+row_nest[2]):
            df = get_s3_object(row_nest[0], row_nest[1], 'FRD_'+row_nest[2])
            df_2hour = get_2_hour(df)
            df_2hour = df_2hour[['Open', 'High', 'Low', 'Close', 'Volume', 'Engine', 'Source', 'Update']]
            save_s3_object(row_nest[0], row_nest[1], twohour, df_2hour)

Downloaded: lc-market-data:Futures/US/CME/ES/FRD_ESC1_hour.txt
10000 frames added
20000 frames added
30000 frames added
40000 frames added
48543 frames added
Saved S3 Object: lc-market-data:Futures/US/CME/ES/FRD_ESC1_2hour.txt
Downloaded: lc-market-data:Futures/US/CME/NQ/FRD_NQC1_hour.txt
10000 frames added
20000 frames added
30000 frames added
40000 frames added
46628 frames added
Saved S3 Object: lc-market-data:Futures/US/CME/NQ/FRD_NQC1_2hour.txt
Downloaded: lc-market-data:Futures/US/CBT/DM/FRD_DMC1_hour.txt
10000 frames added
20000 frames added
30000 frames added
40000 frames added
46105 frames added
Saved S3 Object: lc-market-data:Futures/US/CBT/DM/FRD_DMC1_2hour.txt
Downloaded: lc-market-data:Futures/US/CME/FA/FRD_FAC1_hour.txt
10000 frames added
20000 frames added
30000 frames added
39384 frames added
Saved S3 Object: lc-market-data:Futures/US/CME/FA/FRD_FAC1_2hour.txt
Downloaded: lc-market-data:Futures/US/CME/RTY/FRD_RTYC1_hour.txt
10000 frames added
20000 frames added
30000 fr

In [ ]:
cursor.execute("SELECT [HourRecordId], [DayRecordId] FROM dbo.FuturesStubs WHERE TTSymbol IS NOT NULL AND TTSymbol != ?", 'GC')
for row in cursor.fetchall():
    cursor.execute("SELECT [S3BucketName], [S3PathName], [S3FileName] FROM dbo.DataRecords WHERE Id = ?", row[0])
    for row_nest in cursor.fetchall():
        if check_if_s3_object_exists(row_nest[0], row_nest[1], 'FRD_'+row_nest[2]):
            df = get_s3_object(row_nest[0], row_nest[1], 'FRD_'+row_nest[2])
            cursor.execute("SELECT [S3BucketName], [S3PathName], [S3FileName] FROM dbo.DataRecords WHERE Id = ?", row[1])
            for row_nest2 in cursor.fetchall():
                if cursor.rowcount != 0:
                    save_s3_object(row_nest2[0], row_nest2[1], 'FRD_'+row_nest2[2], get_daily_bars(df))

Downloaded: lc-market-data:Futures/US/CME/ES/FRD_ESC1_hour.txt
Saved S3 Object: lc-market-data:Futures/US/CME/ES/FRD_ESC1_daily.txt
Downloaded: lc-market-data:Futures/US/CME/NQ/FRD_NQC1_hour.txt
Saved S3 Object: lc-market-data:Futures/US/CME/NQ/FRD_NQC1_daily.txt
Downloaded: lc-market-data:Futures/US/CBT/DM/FRD_DMC1_hour.txt
Saved S3 Object: lc-market-data:Futures/US/CBT/DM/FRD_DMC1_daily.txt
Downloaded: lc-market-data:Futures/US/CME/FA/FRD_FAC1_hour.txt
Saved S3 Object: lc-market-data:Futures/US/CME/FA/FRD_FAC1_daily.txt
Downloaded: lc-market-data:Futures/US/CME/RTY/FRD_RTYC1_hour.txt
Saved S3 Object: lc-market-data:Futures/US/CME/RTY/FRD_RTYC1_daily.txt
Downloaded: lc-market-data:Futures/US/NYM/CL/FRD_CLC1_hour.txt
Saved S3 Object: lc-market-data:Futures/US/NYM/CL/FRD_CLC1_daily.txt
Downloaded: lc-market-data:Futures/US/NYM/NG/FRD_NGC1_hour.txt
Saved S3 Object: lc-market-data:Futures/US/NYM/NG/FRD_NGC1_daily.txt
Downloaded: lc-market-data:Futures/US/NYM/XB/FRD_XBC1_hour.txt
Saved S3 